Importing Libraries

In [1]:
pip install catboost


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import sklearn.ensemble
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report
from sklearn.preprocessing import OrdinalEncoder
import lightgbm as lgb
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.utils import resample
import statsmodels
from catboost import CatBoostRegressor
import joblib

Importing datasets

In [3]:
data1 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_train.csv', encoding = 'utf-8')
data2 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/sales_test.csv', encoding = 'utf-8')
data3 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/calendar.csv', encoding = 'utf-8')
data4 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/inventory.csv', encoding = 'utf-8')
data5 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/solution.csv', encoding = 'utf-8')
data6 = pd.read_csv('C:/KUSHAL/Projects/Kaggle/Rohlik Sales Forecasting/inbox/test_weights.csv', encoding = 'utf-8')
print("Datasets successfully imported")

Datasets successfully imported


In [4]:
print(data1.info())
print(data1.isnull().sum()) #info of train set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 14 columns):
 #   Column           Dtype  
---  ------           -----  
 0   unique_id        int64  
 1   date             object 
 2   warehouse        object 
 3   total_orders     float64
 4   sales            float64
 5   sell_price_main  float64
 6   availability     float64
 7   type_0_discount  float64
 8   type_1_discount  float64
 9   type_2_discount  float64
 10  type_3_discount  float64
 11  type_4_discount  float64
 12  type_5_discount  float64
 13  type_6_discount  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 428.0+ MB
None
unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64


In [5]:
pd.set_option('display.max_rows', None)
null_count = data1[data1['total_orders'].isnull() & data1['sales'].isnull()]['date']
print(null_count)

154017     2021-05-23
154099     2021-05-21
154149     2021-05-24
154155     2021-05-22
429815     2021-12-10
429860     2021-12-09
733468     2021-06-27
734198     2021-06-26
738594     2021-05-29
738600     2021-05-23
738632     2021-05-31
738769     2021-05-22
738805     2021-05-30
738816     2021-05-21
738994     2021-05-24
1028203    2021-05-22
1028500    2021-05-23
1028566    2021-05-24
1028753    2021-05-21
1231189    2021-05-21
1231497    2021-05-23
1231500    2021-05-22
1231580    2021-05-29
1231636    2021-05-31
1231739    2021-05-24
1231755    2021-05-30
1451592    2021-06-20
1451661    2021-06-12
1451854    2021-07-04
1451892    2021-06-27
1452131    2021-06-19
1452275    2021-06-13
1452400    2021-07-11
1452632    2021-06-26
3464803    2021-05-23
3464921    2021-05-31
3464993    2021-05-21
3465006    2021-05-24
3465126    2021-05-30
3465135    2021-05-22
3465197    2021-05-29
3744049    2021-07-11
3860150    2021-06-12
3860232    2021-06-13
3862121    2021-12-09
3862202   

In [6]:
pd.set_option('display.max_rows', None)
matching_rows = data3[data3['date'].isin(null_count)][['date', 'holiday', 'warehouse']]
print(matching_rows)

             date  holiday    warehouse
230    2021-06-13        0     Munich_1
470    2021-07-04        0     Prague_1
776    2021-06-19        0   Budapest_1
1079   2021-12-09        0       Brno_1
1091   2021-12-10        0     Prague_2
1118   2021-07-04        0       Brno_1
1255   2021-06-26        0     Prague_3
1568   2021-05-22        0     Prague_1
1740   2021-05-31        0     Munich_1
2315   2021-07-04        0     Prague_2
2422   2021-12-09        0     Prague_2
2432   2021-07-11        0       Brno_1
2497   2021-06-27        0  Frankfurt_1
2548   2021-06-27        0     Prague_3
2553   2021-06-20        0     Munich_1
2561   2021-06-12        0   Budapest_1
3807   2021-06-20        0  Frankfurt_1
4733   2021-05-21        0  Frankfurt_1
5063   2021-05-24        1   Budapest_1
5120   2021-05-30        0     Prague_1
5126   2021-05-29        0       Brno_1
5182   2021-06-19        0     Prague_2
5221   2021-05-30        0   Budapest_1
5382   2021-05-31        0  Frankfurt_1


In [7]:
for date in null_count:
    data1.loc[data1['date'] == date, 'total_orders'] = data1.loc[
    data1['date'] <= date, 'total_orders'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
    data1.loc[data1['date'] == date, 'sales'] = data1.loc[
    data1['date'] <= date, 'sales'].rolling(window = 7, min_periods = 1).mean().iloc[-1]
print(data1.isnull().sum())
print(data1.shape)

unique_id          0
date               0
warehouse          0
total_orders       0
sales              0
sell_price_main    0
availability       0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_discount    0
type_4_discount    0
type_5_discount    0
type_6_discount    0
dtype: int64
(4007419, 14)


In [8]:
print(data2.info())
print(data2.isnull().sum()) #info of test set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        47021 non-null  int64  
 1   date             47021 non-null  object 
 2   warehouse        47021 non-null  object 
 3   total_orders     47021 non-null  float64
 4   sell_price_main  47021 non-null  float64
 5   type_0_discount  47021 non-null  float64
 6   type_1_discount  47021 non-null  float64
 7   type_2_discount  47021 non-null  float64
 8   type_3_discount  47021 non-null  float64
 9   type_4_discount  47021 non-null  float64
 10  type_5_discount  47021 non-null  float64
 11  type_6_discount  47021 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 4.3+ MB
None
unique_id          0
date               0
warehouse          0
total_orders       0
sell_price_main    0
type_0_discount    0
type_1_discount    0
type_2_discount    0
type_3_disco

In [9]:
data2.date.value_counts().sort_index()

date
2024-06-03    3386
2024-06-04    3417
2024-06-05    3450
2024-06-06    3439
2024-06-07    3419
2024-06-08    3362
2024-06-09    3294
2024-06-10    3315
2024-06-11    3323
2024-06-12    3360
2024-06-13    3349
2024-06-14    3353
2024-06-15    3309
2024-06-16    3245
Name: count, dtype: int64

In [10]:
print(data3.info())
print(data3.isnull().sum()) #info of calender set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    23016 non-null  object
 1   holiday_name            930 non-null    object
 2   holiday                 23016 non-null  int64 
 3   shops_closed            23016 non-null  int64 
 4   winter_school_holidays  23016 non-null  int64 
 5   school_holidays         23016 non-null  int64 
 6   warehouse               23016 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.2+ MB
None
date                          0
holiday_name              22086
holiday                       0
shops_closed                  0
winter_school_holidays        0
school_holidays               0
warehouse                     0
dtype: int64


In [11]:
print(data4.info())
print(data4.isnull().sum()) #info of inventory set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5432 entries, 0 to 5431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unique_id            5432 non-null   int64 
 1   product_unique_id    5432 non-null   int64 
 2   name                 5432 non-null   object
 3   L1_category_name_en  5432 non-null   object
 4   L2_category_name_en  5432 non-null   object
 5   L3_category_name_en  5432 non-null   object
 6   L4_category_name_en  5432 non-null   object
 7   warehouse            5432 non-null   object
dtypes: int64(2), object(6)
memory usage: 339.6+ KB
None
unique_id              0
product_unique_id      0
name                   0
L1_category_name_en    0
L2_category_name_en    0
L3_category_name_en    0
L4_category_name_en    0
warehouse              0
dtype: int64


In [12]:
print(data5.info())
print(data5.isnull().sum()) #info of solution set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         47021 non-null  object
 1   sales_hat  47021 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 734.8+ KB
None
id           0
sales_hat    0
dtype: int64


In [13]:
print(data6.info())
print(data6.isnull().sum()) #info of test weights set

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5390 entries, 0 to 5389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   unique_id  5390 non-null   int64  
 1   weight     5390 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 84.3 KB
None
unique_id    0
weight       0
dtype: int64


In [14]:
merge1 = pd.merge(data1, data6, on = 'unique_id', how = 'left')
print(merge1.shape)

(4007419, 15)


In [15]:
merge2 = pd.merge(merge1, data3, on = ['date', 'warehouse'], how = 'left')
print(merge2.shape)

(4007419, 20)


In [16]:
merge3 = pd.merge(merge2, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge3.shape)

(4007419, 26)


In [17]:
merge3['date'] = pd.to_datetime(merge3['date'], errors = 'coerce')
merge3['day'] = merge3['date'].dt.day
merge3['month'] = merge3['date'].dt.month
merge3['year'] = merge3['date'].dt.year
print(merge3.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 29 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [18]:
merge4 = pd.merge(data2, data3, on = ['date', 'warehouse'], how = 'left')
print(merge4.shape)

(47021, 17)


In [19]:
merge5 = pd.merge(merge4, data4, on = ['unique_id', 'warehouse'], how = 'left')
print(merge5.shape)

(47021, 23)


In [20]:
merge5.insert(4, 'sales', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [21]:
merge5.insert(6, 'availability', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [22]:
merge5.insert(14, 'weight', [None] * len(merge5))
print("Column created successfully")

Column created successfully


In [23]:
merge5['date'] = pd.to_datetime(merge5['date'], errors = 'coerce')
merge5['day'] = merge5['date'].dt.day
merge5['month'] = merge5['date'].dt.month
merge5['year'] = merge5['date'].dt.year
print(merge5.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 29 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sales                   0 non-null      object        
 5   sell_price_main         47021 non-null  float64       
 6   availability            0 non-null      object        
 7   type_0_discount         47021 non-null  float64       
 8   type_1_discount         47021 non-null  float64       
 9   type_2_discount         47021 non-null  float64       
 10  type_3_discount         47021 non-null  float64       
 11  type_4_discount         47021 non-null  float64       
 12  type_5_discount         47021 non-null  float6

In [24]:
merge3['log_sales'] = np.log(merge3['sales'] + 1)
merge3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 30 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  weight                  float64       
 15  holiday_name            object        
 16  holiday                 int64         
 17  shops_closed            int64         
 18  wi

In [25]:
merge3 = merge3.sort_values('date')
holiday_dates = merge3[merge3['holiday'] == 1][['date']].rename(columns={'date': 'holiday_date'})

merge3 = pd.merge_asof(merge3, holiday_dates, left_on = 'date', right_on = 'holiday_date', direction = 'backward')
merge3.rename(columns = {'holiday_date': 'last_holiday_date'}, inplace = True)

merge3 = pd.merge_asof(merge3, holiday_dates, left_on = 'date', right_on = 'holiday_date', direction = 'forward')
merge3.rename(columns = {'holiday_date': 'next_holiday_date'}, inplace = True)

merge3['since_last_holiday'] = (merge3['date'] - merge3['last_holiday_date']).dt.days
merge3['until_next_holiday'] = (merge3['next_holiday_date'] - merge3['date']).dt.days

In [26]:
def make_time_features(merge3):
    merge3['date'] = pd.to_datetime(merge3['date'], errors = 'coerce')    
    
    merge3['day'] = merge3['date'].dt.dayofweek.astype(int)
    merge3['week'] = merge3['date'].dt.isocalendar().week.astype(int)
    merge3['month'] = merge3['date'].dt.month.astype(int)
    merge3['day_of_week'] = merge3['date'].dt.strftime('%A')
    merge3['day_in_month'] = merge3['date'].dt.day.astype(int)
    merge3['day_of_year'] = merge3['date'].dt.dayofyear.astype(int)
    merge3['year'] = merge3['date'].dt.year.astype(int)

    merge3['sin_month'] = np.sin(2 * np.pi * merge3['month'] / 12)
    merge3['sin_day'] = np.sin(2 * np.pi * merge3['day'] / 7)
    merge3['sin_week'] = np.sin(2 * np.pi * merge3['week'] / 53)
    merge3['sin_day_year'] = np.sin(2 * np.pi * merge3['day_of_year'] / 365)

    min_date = merge3['date'].min()
    merge3['days_since_start'] = merge3['date'].apply(lambda x: (x - min_date).days)

    merge3['lag_1'] = merge3['log_sales'].shift(1)
    merge3['lag_7'] = merge3['log_sales'].shift(7)
    merge3['lag_30'] = merge3['log_sales'].shift(30)

    merge3['rolling_mean_7'] = merge3['log_sales'].rolling(window = 7).mean()
    merge3['rolling_std_7'] = merge3['log_sales'].rolling(window = 7).std()
    merge3['rolling_mean_30'] = merge3['log_sales'].rolling(window = 30).mean()
    merge3['rolling_std_30'] = merge3['log_sales'].rolling(window = 30).std()

    merge3['rolling_min_7'] = merge3['log_sales'].rolling(window = 7).min()
    merge3['rolling_max_7'] = merge3['log_sales'].rolling(window = 7).max()
    merge3['rolling_min_30'] = merge3['log_sales'].rolling(window = 30).min()
    merge3['rolling_max_30'] = merge3['log_sales'].rolling(window = 30).max()

    merge3['rolling_sum_7'] = merge3['log_sales'].rolling(window = 7).sum()
    merge3['rolling_sum_30'] = merge3['log_sales'].rolling(window = 30).sum()

    merge3['expanding_mean'] = merge3['log_sales'].expanding().mean()

    merge3['is_holiday'] = merge3['holiday'].apply(lambda x: 1 if x else 0)
    merge3['is_weekday'] = merge3['day_of_week'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4] else 0)
    merge3['is_weekend'] = merge3['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)

    merge3 = pd.get_dummies(merge3, columns = ['day_of_week'], drop_first = True)
    merge3 = merge3.replace([np.inf, -np.inf], np.nan).fillna(0)

    return merge3

merge3 = make_time_features(merge3)

In [27]:
def make_time_features(merge5):
    merge5['date'] = pd.to_datetime(merge5['date'], errors = 'coerce')    
    
    merge5['day'] = merge5['date'].dt.dayofweek.astype(int)
    merge5['week'] = merge5['date'].dt.isocalendar().week.astype(int)
    merge5['month'] = merge5['date'].dt.month.astype(int)
    merge5['day_of_week'] = merge5['date'].dt.strftime('%A')
    merge5['day_in_month'] = merge5['date'].dt.day.astype(int)
    merge5['day_of_year'] = merge5['date'].dt.dayofyear.astype(int)
    merge5['year'] = merge5['date'].dt.year.astype(int)

    merge5['sin_month'] = np.sin(2 * np.pi * merge5['month'] / 12)
    merge5['sin_day'] = np.sin(2 * np.pi * merge5['day'] / 7)
    merge5['sin_week'] = np.sin(2 * np.pi * merge5['week'] / 53)
    merge5['sin_day_year'] = np.sin(2 * np.pi * merge5['day_of_year'] / 365)

    min_date = merge5['date'].min()
    merge5['days_since_start'] = merge5['date'].apply(lambda x: (x - min_date).days)

    merge5['is_holiday'] = merge5['holiday'].apply(lambda x: 1 if x else 0)
    merge5['is_weekday'] = merge5['day_of_week'].apply(lambda x: 1 if x in [0, 1, 2, 3, 4] else 0)
    merge5['is_weekend'] = merge5['day_of_week'].apply(lambda x: 1 if x in [5, 6] else 0)

    merge5 = pd.get_dummies(merge5, columns = ['day_of_week'], drop_first = True)
    merge5 = merge5.replace([np.inf, -np.inf], np.nan).fillna(0)

    return merge5

merge5 = make_time_features(merge5)

C:\Users\HP\AppData\Local\Temp\ipykernel_5248\1024312215.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merge5 = merge5.replace([np.inf, -np.inf], np.nan).fillna(0)


In [28]:
merge3 = merge3.sort_values(by = ["product_unique_id", "year", "sin_day_year"]).reset_index(drop = True)
print("Procedure successfull")

Procedure successfull


In [29]:
def assign_around_hol(merge3, original=1):
    merge3["holiday_name"] = merge3["holiday_name"].fillna("")
    for warehouse in merge3["warehouse"].unique():
        warehouse_merge3 = merge3[merge3["warehouse"] == warehouse].copy()

        warehouse_merge3["holiday"] = -2 * warehouse_merge3["holiday"].shift(-1).fillna(0) + warehouse_merge3["holiday"] + 2 * warehouse_merge3["holiday"].shift(1).fillna(0)

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].replace("", np.nan)

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].astype("category")

        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].ffill()
        warehouse_merge3["holiday_name"] = warehouse_merge3["holiday_name"].bfill()

        merge3.loc[merge3["warehouse"] == warehouse, "holiday_name"] = warehouse_merge3["holiday_name"]

assign_around_hol(merge3)
assign_around_hol(merge5)

In [30]:
merge3 = merge3.drop(['date'], axis=1)
print("Column Dropped successfully")

Column Dropped successfully


In [31]:
columns = ['holiday', 'shops_closed', 'winter_school_holidays', 'school_holidays']

def make_lag(merge3, original, columns):
    for column in columns:
        for lag in original:
            merge3[f"{column}_{lag}"] = np.nan  
            
    for column in columns:
        for warehouse in merge3["warehouse"].unique():
            warehouse_data = merge3[merge3["warehouse"] == warehouse].copy()
            
            for lag in original:
                if lag == 0:
                    continue
                warehouse_data[f"{column}_{lag}"] = warehouse_data[column].shift(lag)

            merge3.loc[merge3["warehouse"] == warehouse, [f"{column}_{lag}" for lag in original]] = warehouse_data[[f"{column}_{lag}" for lag in original]]   
    return merge3

In [32]:
drop_columns = ['warehouse', 'holiday_name']
merge3[drop_columns] = merge3[drop_columns].astype(str)
merge5[drop_columns] = merge5[drop_columns].astype(str)

encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value = np.nan)
merge3[drop_columns] = encoder.fit_transform(merge3[drop_columns])
merge5[drop_columns] = encoder.transform(merge5[drop_columns])
warehouse_decoder = {i: encoder.categories_[0][i] for i in range(len(encoder.categories_[0]))}
print(warehouse_decoder)

merge3['warehouse'] = merge3['warehouse'].astype("category")
merge5['warehouse'] = merge5['warehouse'].astype("category")

merge3['warehouse'] = merge3['warehouse'].cat.add_categories('Unknown')
merge5['warehouse'] = merge5['warehouse'].cat.add_categories('Unknown')
merge5 = merge5.fillna({'warehouse': 'Unknown', 'holiday_name': 'Unknown'})

print("Encoding Successful")

{0: 'Brno_1', 1: 'Budapest_1', 2: 'Frankfurt_1', 3: 'Munich_1', 4: 'Prague_1', 5: 'Prague_2', 6: 'Prague_3'}
Encoding Successful


In [33]:
merge3_encoded = pd.get_dummies(merge3, columns=['warehouse'], drop_first=True)
merge5_encoded = pd.get_dummies(merge5, columns=['warehouse'], drop_first=True)

# Align the train and test dataframes to ensure they have the same columns
merge3_encoded, merge5_encoded = merge3_encoded.align(merge5_encoded, join='left', axis=1, fill_value=0)

# Save the data with new features
merge3.to_csv('sales_train_features.csv', index=False)
merge5.to_csv('sales_test_features.csv', index=False)
merge3_encoded.to_csv('sales_train_features_encoded.csv', index=False)
merge5_encoded.to_csv('sales_test_features_encoded.csv', index=False)

print("Encoding completed.")

Encoding completed.


In [34]:
features_to_use = ['weight', 'month', 'year', 'is_weekend', 'day_in_month', 'day_of_year', 'is_weekday', 'holiday',
                   'shops_closed', 'winter_school_holidays', 'school_holidays'] + \
                  [col for col in merge3.columns if col.startswith('warehouse_')]

X = merge3[features_to_use]
X_encoded = merge3_encoded[features_to_use]
y = merge3['log_sales']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Identify categorical feature indices for CatBoost
cat_features_indices = [X_train.columns.get_loc(col) for col in X_train.columns if col.startswith('warehouse_')]

# Initialize models
catboost_model = CatBoostRegressor(cat_features=cat_features_indices, verbose=0)
lightgbm_model = lgb.LGBMRegressor()
xgboost_model = xgb.XGBRegressor()

# Train models
catboost_model.fit(X_train, y_train)
lightgbm_model.fit(X_train_encoded, y_train_encoded)
xgboost_model.fit(X_train_encoded, y_train_encoded)

# Make predictions
y_pred_catboost = catboost_model.predict(X_test)
y_pred_lightgbm = lightgbm_model.predict(X_test_encoded)
y_pred_xgboost = xgboost_model.predict(X_test_encoded)

# Calculate the mean squared error
mse_catboost = MSE(y_test, y_pred_catboost)
mse_lightgbm = MSE(y_test_encoded, y_pred_lightgbm)
mse_xgboost = MSE(y_test_encoded, y_pred_xgboost)

print(f'Mean Squared Error (CatBoost): {mse_catboost}')
print(f'Mean Squared Error (LightGBM): {mse_lightgbm}')
print(f'Mean Squared Error (XGBoost): {mse_xgboost}')

# Save the models
joblib.dump(catboost_model, 'sales_forecasting_catboost_model.pkl')
joblib.dump(lightgbm_model, 'sales_forecasting_lightgbm_model.pkl')
joblib.dump(xgboost_model, 'sales_forecasting_xgboost_model.pkl')

print("Model training completed and models saved.")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.260024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 559
[LightGBM] [Info] Number of data points in the train set: 3205935, number of used features: 9
[LightGBM] [Info] Start training from score 3.784691
Mean Squared Error (CatBoost): 1.3760780540849096
Mean Squared Error (LightGBM): 1.4112556756725456
Mean Squared Error (XGBoost): 1.04523446002659
Model training completed and models saved.


In [35]:
print('The accuracy of the CatBoost model is: ', catboost_model.score(X_test, y_test)) 
print('The accuracy of the training CatBoost model is: ', catboost_model.score(X_train, y_train))

print('The accuracy of the LightGBM model is: ', lightgbm_model.score(X_test_encoded, y_test_encoded)) 
print('The accuracy of the training LightGBM model is: ', lightgbm_model.score(X_train_encoded, y_train_encoded))

print('The accuracy of the XGBoost model is: ', xgboost_model.score(X_test_encoded, y_test_encoded)) 
print('The accuracy of the training XGBoost model is: ', xgboost_model.score(X_train_encoded, y_train_encoded))

The accuracy of the CatBoost model is:  0.1590179252190288
The accuracy of the training CatBoost model is:  0.15946133088588066
The accuracy of the LightGBM model is:  0.13751932701029335
The accuracy of the training LightGBM model is:  0.13797815675488023
The accuracy of the XGBoost model is:  0.3612110576021944
The accuracy of the training XGBoost model is:  0.3616729994934679


In [36]:
# Ensure the feature columns in the test set match those in the train set
X_test_final = merge5[features_to_use]
X_test_final_encoded = merge5_encoded[features_to_use]

# Make predictions on the test set
y_test_pred_catboost = catboost_model.predict(X_test_final)
y_test_pred_lightgbm = lightgbm_model.predict(X_test_final_encoded)
y_test_pred_xgboost = xgboost_model.predict(X_test_final_encoded)

# Convert the predicted log_sales back to original sales values
merge5['predicted_sales_catboost'] = np.exp(y_test_pred_catboost) - 1
merge5['predicted_sales_lightgbm'] = np.exp(y_test_pred_lightgbm) - 1
merge5['predicted_sales_xgboost'] = np.exp(y_test_pred_xgboost) - 1

final1 = merge5['predicted_sales_catboost']
final2 = merge5['predicted_sales_lightgbm']
final3 = merge5['predicted_sales_xgboost']

# Save the predictions to a CSV file
merge5.to_csv('merge5_predictions.csv', index=False)

print("Predictions on test data completed and saved to 'sales_test_predictions.csv'.")

Predictions on test data completed and saved to 'sales_test_predictions.csv'.


In [37]:
# Load the test predictions
merge5_predictions = pd.read_csv('merge5_predictions.csv')

# Create the 'id' column by concatenating 'unique_id' and 'date'
merge5_predictions['id'] = merge5_predictions['unique_id'].astype(str) + '_' + merge5_predictions['date'].astype(str)

# Select the columns for submission
submission = merge5_predictions[['id', 'predicted_sales_catboost']]

# Rename the columns to match the submission format
submission.columns = ['id', 'sales_hat']

# Save the submission file
submission.to_csv('submission.csv', index=False)

print("Submission file created and saved as 'submission.csv'.")

Submission file created and saved as 'submission.csv'.


In [38]:
min_length = min(len(y_test), len(final1))
mae = mean_absolute_error(y_test[:min_length], final1[:min_length])
mse = MSE(y_test[:min_length], final1[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(y_test[:min_length], final1[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

R²: -610439.5211638312
MAE: 1001.5165375329746
MSE: 1004091.5358230489
RMSE: 1002.0436795983741


In [39]:
min_length = min(len(y_test), len(final2))
mae = mean_absolute_error(y_test[:min_length], final2[:min_length])
mse = MSE(y_test[:min_length], final2[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(y_test[:min_length], final2[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

R²: -335651.28750719654
MAE: 743.0284285267242
MSE: 552102.3083838974
RMSE: 743.0358728782194


In [40]:
min_length = min(len(y_test), len(final3))
mae = mean_absolute_error(y_test[:min_length], final3[:min_length])
mse = MSE(y_test[:min_length], final3[:min_length])
rmse = np.sqrt(mse)
r2 = r2_score(y_test[:min_length], final3[:min_length])

print(f"R²: {r2}")
print(f"MAE: {mae}")
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")

R²: -235244.44129382123
MAE: 621.6738264253581
MSE: 386946.71840221743
RMSE: 622.0504146789209


In [41]:
threshold = 0.5
min_length = min(len(y_test), len(final1))
y_pred_class = np.where(final1[:min_length] >= threshold, 1, 0)
val_y_class = np.where(y_test[:min_length] >= threshold, 1, 0)

accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9875
F1 Score: 0.9813
Precision: 0.9751
Recall: 0.9875
Confusion Matrix:
[[    0   588]
 [    0 46433]]


In [42]:
threshold = 0.5
min_length = min(len(y_test), len(final2))
y_pred_class = np.where(final2[:min_length] >= threshold, 1, 0)
val_y_class = np.where(y_test[:min_length] >= threshold, 1, 0)

accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9875
F1 Score: 0.9813
Precision: 0.9751
Recall: 0.9875
Confusion Matrix:
[[    0   588]
 [    0 46433]]


In [43]:
threshold = 0.5
min_length = min(len(y_test), len(final3))
y_pred_class = np.where(final3[:min_length] >= threshold, 1, 0)
val_y_class = np.where(y_test[:min_length] >= threshold, 1, 0)

accuracy = accuracy_score(val_y_class, y_pred_class)
f1 = f1_score(val_y_class, y_pred_class, average='weighted')
precision = precision_score(val_y_class, y_pred_class, average='weighted')
recall = recall_score(val_y_class, y_pred_class, average='weighted')
cm = confusion_matrix(val_y_class, y_pred_class)

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Confusion Matrix:\n{cm}")

C:\Users\HP\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.9875
F1 Score: 0.9813
Precision: 0.9751
Recall: 0.9875
Confusion Matrix:
[[    0   588]
 [    0 46433]]
